# Team Amber : Datathon 2021

## Solution code by:

- Bhavana Balakrishna Rao
- Irene Jin
- Yash Bhatnagar
- Zhenghao Zhang
___________

## Dataset 

Summary

The dataset is comprised of Stathletes-tracked women’s hockey data from the NWHL. The included events have been translated from Stathletes’ raw data to enhance accessibility and interpretability. The various event types include shots, plays, takeaways, puck recoveries, dump ins, dump outs, zone entries, faceoffs and penalties. Event definitions may slightly differ from other sources. For each event, expanded details are provided and the relevant skaters and teams involved are indicated when necessary.

***Contextual Data***

* Date (e.g. ‘2020-12-23’. Format = ‘yyyy-mm-dd’)
* Home Team (e.g. ‘Toronto Maple Leafs’)
* Away Team (e.g. ‘Boston Bruins’)
* Period (range from 1-3 for regulation, 4+ for overtime)
* Clock (e.g. ‘19:34’. Format = ‘mm:ss’)
* Home Team Skaters (range from 3-6 for home skaters currently on the ice)
* Away Team Skaters (range from 3-6 for away skaters currently on the ice)
* Home Team Goals (current goals scored by the home team at the time of the event)
* Away Team Goals (current goals scored by the away team at the time of the event)
* Team (name of the team responsible for the event)
* Player (name of the player responsible for the event)
* Event (type of event, e.g. ‘Play’, ‘Shot’, …)
* X Coordinate (x-coordinate of where an event occurred on the ice, between 0 and 200)
* Y Coordinate (y-coordinate of where an event occurred on the ice, between 0 and 85)
* Coordinates are always from the perspective of the eventing team

* Detail 1-4 (up to 4 supplementary details for each event, varies by event type)
* Player 2 (name of a secondary player involved in an event, varies by event type)
* X Coordinate 2 (x-coordinate of a secondary event detail, varies by event)
* Y Coordinate 2 (y-coordinate of a secondary event detail, varies by event)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [3]:
nwhl = pd.read_csv("/drive/My Drive/Rotman MMA Summer Datathon NWHL.csv")

In [4]:
nwhl.head()

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,Player,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2
0,2021-01-23,Minnesota Whitecaps,Boston Pride,1,20:00,5,5,0,0,Boston Pride,Jillian Dempsey,Faceoff Win,100,43,Backhand,NaN,NaN,NaN,Stephanie Anderson,NaN,NaN
1,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:58,5,5,0,0,Boston Pride,McKenna Brand,Puck Recovery,107,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:57,5,5,0,0,Boston Pride,McKenna Brand,Zone Entry,125,28,Carried,NaN,NaN,NaN,Maddie Rowe,NaN,NaN
3,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:55,5,5,0,0,Boston Pride,McKenna Brand,Shot,131,28,Snapshot,On Net,t,f,NaN,NaN,NaN
4,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:53,5,5,0,0,Boston Pride,Tereza Vanisova,Faceoff Win,169,21,Backhand,NaN,NaN,NaN,Stephanie Anderson,NaN,NaN


In [5]:
nwhl.shape

(26882, 21)

In [6]:
nwhl.describe()

,Period,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,X Coordinate,Y Coordinate,X Coordinate 2,Y Coordinate 2
count,26882.000000,26882.000000,26882.000000,26882.000000,26882.000000,26882.000000,26882.000000,10679.000000,10679.000000
mean,2.010565,4.867086,4.866491,1.408005,0.998847,97.396771,42.192768,108.009739,42.119019
std,0.834753,0.357216,0.376766,1.598554,1.292165,63.188185,27.983077,59.195703,27.196254
min,1.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,5.000000,5.000000,0.000000,0.000000,33.000000,15.000000,58.000000,17.000000
50%,2.000000,5.000000,5.000000,1.000000,1.000000,106.000000,42.000000,112.000000,42.000000
75%,3.000000,5.000000,5.000000,2.000000,1.000000,153.000000,69.000000,161.000000,67.000000
max,4.000000,6.000000,6.000000,6.000000,7.000000,200.000000,85.000000,200.000000,85.000000


In [7]:
nwhl.isnull().sum()

game_date                0
Home Team                0
Away Team                0
Period                   0
Clock                    0
Home Team Skaters        0
Away Team Skaters        0
Home Team Goals          0
Away Team Goals          0
Team                     0
Player                   0
Event                    0
X Coordinate             0
Y Coordinate             0
Detail 1              9421
Detail 2             24897
Detail 3             24897
Detail 4             24897
Player 2             13277
X Coordinate 2       16203
Y Coordinate 2       16203
dtype: int64

In [8]:
nwhl.Event.unique()

array(['Faceoff Win', 'Puck Recovery', 'Zone Entry', 'Shot', 'Play',
       'Takeaway', 'Dump In/Out', 'Incomplete Play', 'Penalty Taken',
       'Goal'], dtype=object)

## Events

* Shot - Shot attempts that are unsuccessful (block, miss or save)
* Goal - Shot attempts that are successful (goal)
* Play - Pass attempts that are successful
* Incomplete Play - Pass attempts that are unsuccessful
* Takeaway - Steals, pass interceptions and won battles that lead to a change in possession
* Puck Recovery - Possession gains initiated by retrieving a loose puck that was created by a missed/blocked/saved shot, an advance (e.g. dump-out/dump-in), a faceoff or a broken play
* Dump In/Out - Actions in which a skater intentionally concedes possession by advancing the puck up ice
* Zone Entry - Attempts to move the puck into the offensive zone from the neutral zone
* Faceoff Win - Faceoffs
* Penalty Taken - Infractions

### Categorizing Goals and Shots 

In [9]:
# Goals made with traffic
# Traffic: an indicator whether there were other players (from either team) between the shooting player and the goalie at the time of the shot.  
# A shot made through traffic is harder to aim and has a high chance of being blocked.  
# However, it is also much harder for the goalie to see and thus may have a higher chance of being successful.

df2 = nwhl.copy()
df2.loc[(df2['Event']=='Goal') & (df2['Detail 3']=='t'),'Event'] = 'Goal+T'
nwhl = df2

In [10]:
# On Net shots -- reaches the net only to be saved (intercepted) by the goalie 
df2 = nwhl.copy()
df2.loc[(df2['Event']=='Shot') & (df2['Detail 2']=='On Net'),'Event']='Shot_OnNet'
# Missed shots -- The shot miss the net
df2.loc[(df2['Event']=='Shot') & (df2['Detail 2']=='Missed'),'Event']='Shot_Missed'
# Blocked shots -- shot is blocked or intercepted before it reaches the net
df2.loc[(df2['Event']=='Shot') & (df2['Detail 2']=='Blocked'),'Event']='Shot_Blocked' 
nwhl=df2

nwhl.Event.unique()

array(['Faceoff Win', 'Puck Recovery', 'Zone Entry', 'Shot_OnNet', 'Play',
       'Takeaway', 'Dump In/Out', 'Incomplete Play', 'Shot_Missed',
       'Shot_Blocked', 'Penalty Taken', 'Goal', 'Goal+T'], dtype=object)

## Players

In [11]:
# Total number of players in the season
len(nwhl.Player.unique())

126

In [12]:
# Event values are converted to columns
players = nwhl.groupby('Player')['Event'].value_counts().unstack().fillna(0)
players.columns.name = None
players.reset_index(level=0, inplace=True)
players.head()

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry
0,Abbie Ives,0.0,0.0,0.0,0.0,3.0,0.0,13.0,22.0,0.0,0.0,0.0,0.0,0.0
1,Allie Olnowich,10.0,0.0,0.0,0.0,7.0,0.0,23.0,31.0,4.0,2.0,4.0,5.0,3.0
2,Allie Thunstrom,24.0,1.0,0.0,0.0,24.0,2.0,45.0,57.0,6.0,10.0,12.0,14.0,32.0
3,Alyson Matteau,20.0,0.0,0.0,0.0,49.0,1.0,130.0,148.0,10.0,5.0,8.0,28.0,17.0
4,Alyssa Wohlfeiler,8.0,0.0,0.0,1.0,27.0,3.0,87.0,52.0,9.0,3.0,12.0,8.0,26.0


In [13]:
players['Shot'] = players['Shot_Blocked'] + players['Shot_Missed']

In this dataset, each row corresponds to one player.

### Goals and Shots

In [14]:
# Get the total number of shots made by the player
players['total_shots'] = (players['Goal'] + players['Goal+T']) + (players['Shot_OnNet'] + players['Shot'])
players.head()

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots
0,Abbie Ives,0.0,0.0,0.0,0.0,3.0,0.0,13.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Allie Olnowich,10.0,0.0,0.0,0.0,7.0,0.0,23.0,31.0,4.0,2.0,4.0,5.0,3.0,6.0,10.0
2,Allie Thunstrom,24.0,1.0,0.0,0.0,24.0,2.0,45.0,57.0,6.0,10.0,12.0,14.0,32.0,16.0,28.0
3,Alyson Matteau,20.0,0.0,0.0,0.0,49.0,1.0,130.0,148.0,10.0,5.0,8.0,28.0,17.0,15.0,23.0
4,Alyssa Wohlfeiler,8.0,0.0,0.0,1.0,27.0,3.0,87.0,52.0,9.0,3.0,12.0,8.0,26.0,12.0,25.0


In [15]:
players.sort_values(by=['Goal'], ascending=False).head(n=5)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots
87,Mikyla Grant-Mentis,19.0,19.0,4.0,1.0,44.0,3.0,104.0,102.0,12.0,11.0,26.0,19.0,58.0,23.0,54.0
9,Autumn MacDougall,28.0,0.0,3.0,0.0,48.0,0.0,71.0,77.0,5.0,2.0,11.0,7.0,33.0,7.0,21.0
98,Samantha Davis,21.0,5.0,3.0,1.0,48.0,1.0,119.0,111.0,3.0,12.0,19.0,18.0,54.0,15.0,38.0
44,Jillian Dempsey,10.0,70.0,3.0,0.0,36.0,0.0,80.0,72.0,4.0,6.0,14.0,22.0,19.0,10.0,27.0
116,Taylor Woods,31.0,2.0,3.0,1.0,46.0,5.0,157.0,175.0,20.0,12.0,10.0,14.0,30.0,32.0,46.0


In [16]:
# Find the players goal score percentage
# OnNet %: Tells us that actually how many shots reached the goal(either resulted in a 'goal' or were saved by a goalie-'on net')
players['OnNet_pct'] = (players['Goal'] + players['Goal+T'] + players['Shot_OnNet'])/players['total_shots']
players.fillna(0, inplace=True)
players.head(3)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct
0,Abbie Ives,0.0,0.0,0.0,0.0,3.0,0.0,13.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Allie Olnowich,10.0,0.0,0.0,0.0,7.0,0.0,23.0,31.0,4.0,2.0,4.0,5.0,3.0,6.0,10.0,0.400000
2,Allie Thunstrom,24.0,1.0,0.0,0.0,24.0,2.0,45.0,57.0,6.0,10.0,12.0,14.0,32.0,16.0,28.0,0.428571


In [17]:
players.sort_values(by=['OnNet_pct'], ascending=False).head(n=5)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct
73,Maeve Reilly,3.0,0.0,0.0,0.0,3.0,0.0,3.0,3.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,1.000000
53,Kayla Meneghin,5.0,5.0,0.0,0.0,17.0,4.0,12.0,17.0,0.0,0.0,2.0,2.0,10.0,0.0,2.0,1.000000
50,Katelynn Russ,19.0,6.0,2.0,0.0,30.0,2.0,40.0,56.0,2.0,3.0,21.0,11.0,21.0,5.0,28.0,0.821429
45,Jonna Curtis,24.0,26.0,1.0,0.0,37.0,0.0,57.0,72.0,5.0,0.0,15.0,10.0,33.0,5.0,21.0,0.761905
36,Haley Mack,8.0,6.0,2.0,0.0,28.0,1.0,62.0,59.0,1.0,3.0,9.0,4.0,31.0,4.0,15.0,0.733333


In [18]:
# Shot Score -- Tells us the accuracy of the shots (1- missed)
players['Shot Score'] = 1 - (players['Shot_Missed']/players['total_shots'])
players.sort_values(by=['Shot Score'], ascending=False).head(n=5)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score
53,Kayla Meneghin,5.0,5.0,0.0,0.0,17.0,4.0,12.0,17.0,0.0,0.0,2.0,2.0,10.0,0.0,2.0,1.000000,1.0
97,Rose Alleva,0.0,0.0,0.0,0.0,2.0,1.0,5.0,7.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,0.000000,1.0
23,Chelsey Brodt-Rosenthal,17.0,0.0,0.0,0.0,27.0,3.0,33.0,74.0,2.0,0.0,1.0,4.0,4.0,2.0,3.0,0.333333,1.0
39,Hunter Accursi,11.0,3.0,0.0,0.0,14.0,1.0,19.0,34.0,2.0,0.0,5.0,3.0,9.0,2.0,7.0,0.714286,1.0
42,Jayne Lewis,6.0,0.0,0.0,0.0,11.0,0.0,10.0,12.0,1.0,0.0,2.0,0.0,2.0,1.0,3.0,0.666667,1.0


In [19]:
# Threshold value to calculate Shoot Score
shot_sc = players['total_shots'].median()
shot_sc

14.0

In [20]:
# Cleaning the data according to the threshold value
players.loc[(players['total_shots']<shot_sc),'Shot Score'] = 0
players.sort_values(by=['Shot Score'], ascending=False).head(n=5)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score
45,Jonna Curtis,24.0,26.0,1.0,0.0,37.0,0.0,57.0,72.0,5.0,0.0,15.0,10.0,33.0,5.0,21.0,0.761905,1.000000
120,Theresa Knutson,5.0,2.0,1.0,0.0,10.0,0.0,36.0,32.0,6.0,1.0,9.0,5.0,14.0,7.0,17.0,0.588235,0.941176
56,Kiira Dosdall,16.0,0.0,0.0,0.0,34.0,0.0,91.0,88.0,4.0,1.0,12.0,8.0,12.0,5.0,17.0,0.705882,0.941176
86,Meghara McManus,26.0,2.0,0.0,1.0,28.0,2.0,31.0,51.0,7.0,2.0,15.0,17.0,37.0,9.0,25.0,0.640000,0.920000
9,Autumn MacDougall,28.0,0.0,3.0,0.0,48.0,0.0,71.0,77.0,5.0,2.0,11.0,7.0,33.0,7.0,21.0,0.666667,0.904762


In [21]:
# Calculate goal normalised - it will be in the range of 0-1
players['goal_normalised'] = players['Goal']/max(players['Goal'])
players['goal+T_normalised'] = players['Goal+T']/max(players['Goal+T'])

#### Calculating goal_avg using normalised scores

In [22]:
# Weight of Goal with traffic > Goal > Shots on Net = Shot Accuracy - 3/7 : 2/7 : 1/7 : 1/7
players['goal_avg'] = (2*players['goal_normalised'] + 3*players['goal+T_normalised']+players['OnNet_pct']+players['Shot Score'])/7
players.sort_values(by=['goal_avg'], ascending=False).head(5)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg
87,Mikyla Grant-Mentis,19.0,19.0,4.0,1.0,44.0,3.0,104.0,102.0,12.0,11.0,26.0,19.0,58.0,23.0,54.0,0.574074,0.796296,1.00,0.5,0.695767
80,McKenna Brand,24.0,6.0,1.0,2.0,34.0,2.0,142.0,111.0,20.0,12.0,23.0,7.0,55.0,32.0,58.0,0.448276,0.793103,0.25,1.0,0.677340
76,Mallory Souliotis,27.0,0.0,1.0,2.0,84.0,0.0,175.0,196.0,9.0,9.0,14.0,22.0,18.0,18.0,35.0,0.485714,0.742857,0.25,1.0,0.675510
98,Samantha Davis,21.0,5.0,3.0,1.0,48.0,1.0,119.0,111.0,3.0,12.0,19.0,18.0,54.0,15.0,38.0,0.605263,0.684211,0.75,0.5,0.612782
116,Taylor Woods,31.0,2.0,3.0,1.0,46.0,5.0,157.0,175.0,20.0,12.0,10.0,14.0,30.0,32.0,46.0,0.304348,0.739130,0.75,0.5,0.577640


Consider top 10% of players in every category

In [23]:
ratio = 0.1
threshold_pct = np.int(np.floor(ratio*len(nwhl.Player.unique())))
threshold_pct

12

In [24]:
goal_avg_thres = players.sort_values(by=['goal_avg'], ascending=False)['goal_avg'][threshold_pct-1:threshold_pct].values[0]
goal_avg_thres

0.4387755102040816

#### Consider players with Goals > goal_avg_thres as excellent goal scorers 

In [25]:
# use this feature for goal scoring constraint in the Linear programming problem
players['goal_scorers'] = (players['goal_avg'] >= goal_avg_thres).astype(int)

### Play and Incomplete Play

In [26]:
# Get the total number of passes made by the player
players['total_passes'] = players['Play'] + players['Incomplete Play']
players.head(n=3)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg,goal_scorers,total_passes
0,Abbie Ives,0.0,0.0,0.0,0.0,3.0,0.0,13.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0,16.0
1,Allie Olnowich,10.0,0.0,0.0,0.0,7.0,0.0,23.0,31.0,4.0,2.0,4.0,5.0,3.0,6.0,10.0,0.400000,0.000000,0.0,0.0,0.057143,0,30.0
2,Allie Thunstrom,24.0,1.0,0.0,0.0,24.0,2.0,45.0,57.0,6.0,10.0,12.0,14.0,32.0,16.0,28.0,0.428571,0.642857,0.0,0.0,0.153061,0,69.0


In [27]:
# Find the players successful pass score percentage
players['pass_pct'] = players['Play']/players['total_passes']
players.head(n=3)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg,goal_scorers,total_passes,pass_pct
0,Abbie Ives,0.0,0.0,0.0,0.0,3.0,0.0,13.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0,16.0,0.812500
1,Allie Olnowich,10.0,0.0,0.0,0.0,7.0,0.0,23.0,31.0,4.0,2.0,4.0,5.0,3.0,6.0,10.0,0.400000,0.000000,0.0,0.0,0.057143,0,30.0,0.766667
2,Allie Thunstrom,24.0,1.0,0.0,0.0,24.0,2.0,45.0,57.0,6.0,10.0,12.0,14.0,32.0,16.0,28.0,0.428571,0.642857,0.0,0.0,0.153061,0,69.0,0.652174


In [28]:
players.sort_values(by=['Play'], ascending=False).head(n=3)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg,goal_scorers,total_passes,pass_pct
48,Kaleigh Fratkin,41.0,0.0,0.0,0.0,90.0,5.0,242.0,233.0,16.0,11.0,9.0,16.0,23.0,27.0,36.0,0.250000,0.694444,0.00,0.0,0.134921,0,332.0,0.728916
76,Mallory Souliotis,27.0,0.0,1.0,2.0,84.0,0.0,175.0,196.0,9.0,9.0,14.0,22.0,18.0,18.0,35.0,0.485714,0.742857,0.25,1.0,0.675510,1,259.0,0.675676
104,Sarah Steele,11.0,0.0,0.0,0.0,55.0,1.0,158.0,141.0,8.0,4.0,5.0,11.0,11.0,12.0,17.0,0.294118,0.764706,0.00,0.0,0.151261,0,213.0,0.741784


**Kaleigh Fratkin** has scored the most number of successful passes in this season.  

In [29]:
players.sort_values(by=['pass_pct'], ascending=False).head(n=3)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg,goal_scorers,total_passes,pass_pct
99,Samantha Ridgewell,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,10.0,1.0
123,Victoria Hanson,1.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,12.0,1.0
22,Caty Flagg,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1.0


In [30]:
# Calculate Play normalised - it will be in the range of 0-1
players['play_normalised'] = players['Play']/max(players['Play'])

# Calculate average of play_normalised and pass_pct - this will take both the fields into account 
players['pass_avg'] = (2*players['play_normalised'] + players['pass_pct'])/3
players.sort_values(by=['pass_avg'], ascending=False).head()

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg,goal_scorers,total_passes,pass_pct,play_normalised,pass_avg
48,Kaleigh Fratkin,41.0,0.0,0.0,0.0,90.0,5.0,242.0,233.0,16.0,11.0,9.0,16.0,23.0,27.0,36.0,0.250000,0.694444,0.00,0.0,0.134921,0,332.0,0.728916,1.000000,0.909639
76,Mallory Souliotis,27.0,0.0,1.0,2.0,84.0,0.0,175.0,196.0,9.0,9.0,14.0,22.0,18.0,18.0,35.0,0.485714,0.742857,0.25,1.0,0.675510,1,259.0,0.675676,0.723140,0.707319
116,Taylor Woods,31.0,2.0,3.0,1.0,46.0,5.0,157.0,175.0,20.0,12.0,10.0,14.0,30.0,32.0,46.0,0.304348,0.739130,0.75,0.5,0.577640,1,203.0,0.773399,0.648760,0.690307
104,Sarah Steele,11.0,0.0,0.0,0.0,55.0,1.0,158.0,141.0,8.0,4.0,5.0,11.0,11.0,12.0,17.0,0.294118,0.764706,0.00,0.0,0.151261,0,213.0,0.741784,0.652893,0.682523
108,Shiann Darkangelo,25.0,79.0,0.0,0.0,63.0,5.0,154.0,141.0,8.0,6.0,22.0,23.0,37.0,14.0,36.0,0.611111,0.833333,0.00,0.0,0.206349,0,217.0,0.709677,0.636364,0.660802


In [31]:
pass_avg_thres = players.sort_values(by=['pass_avg'], ascending=False)['pass_avg'][threshold_pct-1:threshold_pct].values[0]
pass_avg_thres

0.5969175787357606

#### Consider players with pass_pct > pass_avg_thres as excellent passers



In [32]:
# use this feature for passers constraint in the Linear programming problem
players['passers'] = (players['pass_avg'] >= pass_avg_thres).astype(int)
players.head(n=3)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg,goal_scorers,total_passes,pass_pct,play_normalised,pass_avg,passers
0,Abbie Ives,0.0,0.0,0.0,0.0,3.0,0.0,13.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0,16.0,0.812500,0.053719,0.306646,0
1,Allie Olnowich,10.0,0.0,0.0,0.0,7.0,0.0,23.0,31.0,4.0,2.0,4.0,5.0,3.0,6.0,10.0,0.400000,0.000000,0.0,0.0,0.057143,0,30.0,0.766667,0.095041,0.318916,0
2,Allie Thunstrom,24.0,1.0,0.0,0.0,24.0,2.0,45.0,57.0,6.0,10.0,12.0,14.0,32.0,16.0,28.0,0.428571,0.642857,0.0,0.0,0.153061,0,69.0,0.652174,0.185950,0.341358,0


### Faceoff

In [33]:
players.sort_values(by=['Faceoff Win'], ascending=False).head(n=3)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg,goal_scorers,total_passes,pass_pct,play_normalised,pass_avg,passers
108,Shiann Darkangelo,25.0,79.0,0.0,0.0,63.0,5.0,154.0,141.0,8.0,6.0,22.0,23.0,37.0,14.0,36.0,0.611111,0.833333,0.00,0.0,0.206349,0,217.0,0.709677,0.636364,0.660802,1
44,Jillian Dempsey,10.0,70.0,3.0,0.0,36.0,0.0,80.0,72.0,4.0,6.0,14.0,22.0,19.0,10.0,27.0,0.629630,0.777778,0.75,0.0,0.415344,0,116.0,0.689655,0.330579,0.450271,0
63,Lexie Laing,23.0,55.0,1.0,0.0,46.0,1.0,75.0,74.0,6.0,4.0,13.0,17.0,24.0,10.0,24.0,0.583333,0.833333,0.25,0.0,0.273810,0,121.0,0.619835,0.309917,0.413223,0


In [34]:
# Faceoff Loss - number of times the player lost in the Faceoff
faceoff_loss = nwhl[(nwhl['Event'] == 'Faceoff Win')].groupby('Player 2')['Event'].count()
players['Faceoff Loss'] = players['Player'].apply(lambda x: faceoff_loss[x] if(x in faceoff_loss) else 0)
players['faceoff_pct'] = (players['Faceoff Win'] / (players['Faceoff Win'] + players['Faceoff Loss'])).fillna(0)
players.sort_values(by=['faceoff_pct'], ascending=False).head()

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg,goal_scorers,total_passes,pass_pct,play_normalised,pass_avg,passers,Faceoff Loss,faceoff_pct
85,Meghan Lorence,15.0,1.0,0.0,0.0,29.0,1.0,73.0,53.0,4.0,3.0,14.0,11.0,25.0,7.0,21.0,0.666667,0.857143,0.0,0.0,0.217687,0,102.0,0.715686,0.301653,0.439664,0,0,1.0
93,Paige Voight,15.0,1.0,0.0,0.0,21.0,0.0,29.0,36.0,2.0,2.0,1.0,8.0,9.0,4.0,5.0,0.200000,0.000000,0.0,0.0,0.028571,0,50.0,0.580000,0.119835,0.273223,0,0,1.0
16,Brooke White,2.0,1.0,0.0,0.0,4.0,0.0,7.0,9.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0,11.0,0.636364,0.028926,0.231405,0,0,1.0
122,Tori Sullivan,10.0,1.0,0.0,0.0,16.0,0.0,86.0,58.0,5.0,4.0,4.0,14.0,21.0,9.0,13.0,0.307692,0.000000,0.0,0.0,0.043956,0,102.0,0.843137,0.355372,0.517960,0,0,1.0
101,Sara Bustad,8.0,1.0,0.0,0.0,20.0,1.0,27.0,45.0,5.0,4.0,3.0,2.0,6.0,9.0,12.0,0.250000,0.000000,0.0,0.0,0.035714,0,47.0,0.574468,0.111570,0.265870,0,0,1.0


In [35]:
# Calculate faceoff win normalised - it will be in the range of 0-1
players['faceoff_win_normalised'] = players['Faceoff Win']/max(players['Faceoff Win'])

# Calculate average of faceoff_win_normalised and faceoff_pct - this will take both the fields into account 
players['faceoff_avg'] = (2*players['faceoff_win_normalised'] + players['faceoff_pct'])/3
players.sort_values(by=['faceoff_avg'], ascending=False).head(5)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg,goal_scorers,total_passes,pass_pct,play_normalised,pass_avg,passers,Faceoff Loss,faceoff_pct,faceoff_win_normalised,faceoff_avg
108,Shiann Darkangelo,25.0,79.0,0.0,0.0,63.0,5.0,154.0,141.0,8.0,6.0,22.0,23.0,37.0,14.0,36.0,0.611111,0.833333,0.00,0.0,0.206349,0,217.0,0.709677,0.636364,0.660802,1,46,0.632000,1.000000,0.877333
44,Jillian Dempsey,10.0,70.0,3.0,0.0,36.0,0.0,80.0,72.0,4.0,6.0,14.0,22.0,19.0,10.0,27.0,0.629630,0.777778,0.75,0.0,0.415344,0,116.0,0.689655,0.330579,0.450271,0,27,0.721649,0.886076,0.831267
63,Lexie Laing,23.0,55.0,1.0,0.0,46.0,1.0,75.0,74.0,6.0,4.0,13.0,17.0,24.0,10.0,24.0,0.583333,0.833333,0.25,0.0,0.273810,0,121.0,0.619835,0.309917,0.413223,0,39,0.585106,0.696203,0.659170
119,Tereza Vanisova,5.0,51.0,0.0,0.0,35.0,5.0,138.0,116.0,6.0,10.0,14.0,17.0,62.0,16.0,30.0,0.466667,0.666667,0.00,0.0,0.161905,0,173.0,0.797688,0.570248,0.646061,1,26,0.662338,0.645570,0.651159
10,Breanne Wilson-Bennett,21.0,45.0,1.0,1.0,35.0,1.0,95.0,107.0,6.0,8.0,17.0,25.0,29.0,14.0,33.0,0.575758,0.757576,0.25,0.5,0.476190,1,130.0,0.730769,0.392562,0.505298,0,32,0.584416,0.569620,0.574552


In [36]:
faceoff_avg_thres = players.sort_values(by=['faceoff_avg'], ascending=False)['faceoff_avg'][threshold_pct-1:threshold_pct].values[0]
faceoff_avg_thres

0.3845815752461322

#### Consider players with Faceoff Win > faceoff_avg_thres as Faceoff Specialists

In [37]:
# use this feature for faceoff constraint in the Linear programming problem
players['faceoff_specialist'] = (players['faceoff_avg'] >= faceoff_avg_thres).astype(int)

### Takeaway  Specialist

In [38]:
players.sort_values(by=['Takeaway'], ascending=False).head(n=3)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg,goal_scorers,total_passes,pass_pct,play_normalised,pass_avg,passers,Faceoff Loss,faceoff_pct,faceoff_win_normalised,faceoff_avg,faceoff_specialist
25,Dominique Kremer,36.0,0.0,0.0,0.0,58.0,2.0,99.0,134.0,4.0,7.0,5.0,29.0,32.0,11.0,16.0,0.312500,0.562500,0.00,0.0,0.125000,0,157.0,0.630573,0.409091,0.482918,0,0,0.000000,0.00000,0.000000,0
3,Alyson Matteau,20.0,0.0,0.0,0.0,49.0,1.0,130.0,148.0,10.0,5.0,8.0,28.0,17.0,15.0,23.0,0.347826,0.782609,0.00,0.0,0.161491,0,179.0,0.726257,0.537190,0.600212,1,0,0.000000,0.00000,0.000000,0
10,Breanne Wilson-Bennett,21.0,45.0,1.0,1.0,35.0,1.0,95.0,107.0,6.0,8.0,17.0,25.0,29.0,14.0,33.0,0.575758,0.757576,0.25,0.5,0.476190,1,130.0,0.730769,0.392562,0.505298,0,32,0.584416,0.56962,0.574552,1


In [39]:
takeaway_thres = players.sort_values(by=['Takeaway'], ascending=False)['Takeaway'][threshold_pct-1:threshold_pct].values[0]
takeaway_thres

20.0

#### Consider players with Takeaway > takeaway_thres as Takeaway Specialists

In [40]:
# use this feature for takeoff constraint in the Linear programming problem
players['takeaway_specialist'] = (players['Takeaway'] >= takeaway_thres).astype(int)

## Linear Programming Problem

The selection of the new team is a constrained optimization problem. 

In [41]:
!pip install pulp

     |████████████████████████████████| 40.6MB 72kB/s 


#### Transforming player selection into an LPP
*The coaches would like to select five top players.  The players should, ideally, excel in all areas of the game.  Since scoring goals is the ultimate objective, at least three should be excellent goal scorers and at least two should be excellent passers.  In addition, you need at least two faceoff specialists and one takeaway specialist.  It would be especially beneficial if the passers were familiar with the shooting specialists (i.e., had a good track record of completing passes to these players).*

#### Objective Function:
* Maximize the number of goals scored by the team

#### Decision Variable
* The Players

#### Constraints:
* Select five top players
* At least three should be excellent goal scorers
* At least two should be excellent passers
* At least two faceoff specialists 
* At least one takeaway specialist

In [42]:
from pulp import *

In [43]:
prob = pulp.LpProblem('PlayerSelection', LpMaximize)

#### Decision variables.
The players are decision variables. It can take values 1 or 0 depending on whether the player is selected or not for top 5.

In [44]:
decision_variables = []
for rownum, row in players.iterrows():
    variable = str('x' + str(rownum))
    variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
    decision_variables.append(variable)

print ("Total number of decision_variables: " + str(len(decision_variables)))

Total number of decision_variables: 126


#### Optimization function

In [45]:
total_points = ""
for rownum, row in players.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            formula = row['Goal']*player
            total_points += formula

prob += total_points
display ("Optimization function: " + str(total_points))

'Optimization function: x10 + x105 + x115 + 3.0*x116 + x117 + x120 + 2.0*x14 + 2.0*x24 + x29 + x32 + 2.0*x36 + 3.0*x44 + x45 + x46 + x5 + 2.0*x50 + x51 + x58 + x60 + x63 + x64 + x69 + x7 + x75 + x76 + x8 + x80 + x81 + x82 + x84 + 4.0*x87 + 3.0*x9 + 2.0*x91 + x96 + 3.0*x98'

#### Constraints

In [46]:
# Select top 5 players
req_players = 5
total_players = ""
for rownum, row in players.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            formula = 1*player
            total_players += formula
prob += (total_players == req_players)
#print((total_players))
display (str(total_players) + " == 5")

'x0 + x1 + x10 + x100 + x101 + x102 + x103 + x104 + x105 + x106 + x107 + x108 + x109 + x11 + x110 + x111 + x112 + x113 + x114 + x115 + x116 + x117 + x118 + x119 + x12 + x120 + x121 + x122 + x123 + x124 + x125 + x13 + x14 + x15 + x16 + x17 + x18 + x19 + x2 + x20 + x21 + x22 + x23 + x24 + x25 + x26 + x27 + x28 + x29 + x3 + x30 + x31 + x32 + x33 + x34 + x35 + x36 + x37 + x38 + x39 + x4 + x40 + x41 + x42 + x43 + x44 + x45 + x46 + x47 + x48 + x49 + x5 + x50 + x51 + x52 + x53 + x54 + x55 + x56 + x57 + x58 + x59 + x6 + x60 + x61 + x62 + x63 + x64 + x65 + x66 + x67 + x68 + x69 + x7 + x70 + x71 + x72 + x73 + x74 + x75 + x76 + x77 + x78 + x79 + x8 + x80 + x81 + x82 + x83 + x84 + x85 + x86 + x87 + x88 + x89 + x9 + x90 + x91 + x92 + x93 + x94 + x95 + x96 + x97 + x98 + x99 == 5'

In [47]:
# At least three should be excellent goal scorers
req_goal_scorer = 3
total_goal_scorer = ""
for rownum, row in players.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['goal_scorers'] == 1:
                formula = 1*player
                total_goal_scorer += formula
prob += (total_goal_scorer >= req_goal_scorer)
print(total_goal_scorer)

x10 + x116 + x58 + x61 + x69 + x76 + x8 + x80 + x82 + x87 + x9 + x98


In [48]:
# At least two should be excellent passers
req_passers = 2
total_passers = ""
for rownum, row in players.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['passers'] == 1:
                formula = 1*player
                total_passers += formula
prob += (total_passers >= req_passers)
#print((total_passers))
display (str(total_passers) + " >= 2")

'x104 + x108 + x113 + x116 + x119 + x3 + x48 + x60 + x64 + x76 + x78 + x80 >= 2'

In [49]:
# At least two faceoff specialists
req_faceoff_specialist = 2
total_faceoff_specialist = ""
for rownum, row in players.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['faceoff_specialist'] == 1:
                formula = 1*player
                total_faceoff_specialist += formula
prob += (total_faceoff_specialist >= req_faceoff_specialist)
print((total_faceoff_specialist))

x10 + x103 + x108 + x110 + x119 + x19 + x21 + x32 + x44 + x46 + x52 + x63


In [50]:
# At least one takeaway specialist
req_takeoff_specialist = 1
total_takeoff_specialist = ""
for rownum, row in players.iterrows():
    for i, player in enumerate(decision_variables):
        if rownum == i:
            if row['takeaway_specialist'] == 1:
                formula = 1*player
                total_takeoff_specialist += formula
prob += (total_takeoff_specialist >= req_takeoff_specialist)
print((total_takeoff_specialist))

x10 + x108 + x111 + x21 + x25 + x3 + x44 + x46 + x62 + x76 + x78 + x88


#### Solve the LPP problem

In [51]:
prob.writeLP('PlayerSelection.lp')
optimization_result = prob.solve()
assert optimization_result == LpStatusOptimal
print("Status:", LpStatus[prob.status])
print("Optimal Solution to the problem: ", value(prob.objective))
print ("Individual decision_variables: ")
for v in prob.variables():
    print(v.name, "=", v.varValue)

Status: Optimal
Optimal Solution to the problem:  13.0
Individual decision_variables: 
x0 = 0.0
x1 = 0.0
x10 = 0.0
x100 = 0.0
x101 = 0.0
x102 = 0.0
x103 = 0.0
x104 = 0.0
x105 = 0.0
x106 = 0.0
x107 = 0.0
x108 = 0.0
x109 = 0.0
x11 = 0.0
x110 = 0.0
x111 = 0.0
x112 = 0.0
x113 = 0.0
x114 = 0.0
x115 = 0.0
x116 = 1.0
x117 = 0.0
x118 = 0.0
x119 = 1.0
x12 = 0.0
x120 = 0.0
x121 = 0.0
x122 = 0.0
x123 = 0.0
x124 = 0.0
x125 = 0.0
x13 = 0.0
x14 = 0.0
x15 = 0.0
x16 = 0.0
x17 = 0.0
x18 = 0.0
x19 = 0.0
x2 = 0.0
x20 = 0.0
x21 = 0.0
x22 = 0.0
x23 = 0.0
x24 = 0.0
x25 = 0.0
x26 = 0.0
x27 = 0.0
x28 = 0.0
x29 = 0.0
x3 = 0.0
x30 = 0.0
x31 = 0.0
x32 = 0.0
x33 = 0.0
x34 = 0.0
x35 = 0.0
x36 = 0.0
x37 = 0.0
x38 = 0.0
x39 = 0.0
x4 = 0.0
x40 = 0.0
x41 = 0.0
x42 = 0.0
x43 = 0.0
x44 = 1.0
x45 = 0.0
x46 = 0.0
x47 = 0.0
x48 = 0.0
x49 = 0.0
x5 = 0.0
x50 = 0.0
x51 = 0.0
x52 = 0.0
x53 = 0.0
x54 = 0.0
x55 = 0.0
x56 = 0.0
x57 = 0.0
x58 = 0.0
x59 = 0.0
x6 = 0.0
x60 = 0.0
x61 = 0.0
x62 = 0.0
x63 = 0.0
x64 = 0.0
x65 = 0.0
x66 

#### Top 5 players selected my the LPP model

In [52]:
variable_name = []
variable_value = []

for v in prob.variables():
    variable_name.append(v.name)
    variable_value.append(v.varValue)

df = pd.DataFrame({'variable': variable_name, 'value': variable_value})
for rownum, row in df.iterrows():
    value = re.findall(r'(\d+)', row['variable'])
    df.loc[rownum, 'variable'] = int(value[0])

df = df.sort_index()

#append results
for rownum, row in players.iterrows():
    for results_rownum, results_row in df.iterrows():
        if rownum == results_row['variable']:
            players.loc[rownum, 'decision'] = results_row['value']
 
players[players.decision==1].sort_values('Player').head(5)

,Player,Dump In/Out,Faceoff Win,Goal,Goal+T,Incomplete Play,Penalty Taken,Play,Puck Recovery,Shot_Blocked,Shot_Missed,Shot_OnNet,Takeaway,Zone Entry,Shot,total_shots,OnNet_pct,Shot Score,goal_normalised,goal+T_normalised,goal_avg,goal_scorers,total_passes,pass_pct,play_normalised,pass_avg,passers,Faceoff Loss,faceoff_pct,faceoff_win_normalised,faceoff_avg,faceoff_specialist,takeaway_specialist,decision
44,Jillian Dempsey,10.0,70.0,3.0,0.0,36.0,0.0,80.0,72.0,4.0,6.0,14.0,22.0,19.0,10.0,27.0,0.629630,0.777778,0.75,0.0,0.415344,0,116.0,0.689655,0.330579,0.450271,0,27,0.721649,0.886076,0.831267,1,1,1.0
87,Mikyla Grant-Mentis,19.0,19.0,4.0,1.0,44.0,3.0,104.0,102.0,12.0,11.0,26.0,19.0,58.0,23.0,54.0,0.574074,0.796296,1.00,0.5,0.695767,1,148.0,0.702703,0.429752,0.520736,0,16,0.542857,0.240506,0.341290,0,0,1.0
98,Samantha Davis,21.0,5.0,3.0,1.0,48.0,1.0,119.0,111.0,3.0,12.0,19.0,18.0,54.0,15.0,38.0,0.605263,0.684211,0.75,0.5,0.612782,1,167.0,0.712575,0.491736,0.565349,0,3,0.625000,0.063291,0.250527,0,0,1.0
116,Taylor Woods,31.0,2.0,3.0,1.0,46.0,5.0,157.0,175.0,20.0,12.0,10.0,14.0,30.0,32.0,46.0,0.304348,0.739130,0.75,0.5,0.577640,1,203.0,0.773399,0.648760,0.690307,1,1,0.666667,0.025316,0.239100,0,0,1.0
119,Tereza Vanisova,5.0,51.0,0.0,0.0,35.0,5.0,138.0,116.0,6.0,10.0,14.0,17.0,62.0,16.0,30.0,0.466667,0.666667,0.00,0.0,0.161905,0,173.0,0.797688,0.570248,0.646061,1,26,0.662338,0.645570,0.651159,1,0,1.0


In [53]:
#players[players.decision==1]['Player', 'Goal']
df_top5 = players[(players['decision'] == 1)]
df_top5[['Player', 'goal_scorers', 'passers', 'faceoff_specialist', 'takeaway_specialist', 'Goal', 'Goal+T', 'goal_normalised', 'goal+T_normalised', 'OnNet_pct', 'goal_avg', 'Shot_OnNet', 'Shot Score','Play', 'play_normalised', 'pass_avg', 'Faceoff Win', 'faceoff_win_normalised', 'faceoff_pct', 'Takeaway']]
#df_top5

,Player,goal_scorers,passers,faceoff_specialist,takeaway_specialist,Goal,Goal+T,goal_normalised,goal+T_normalised,OnNet_pct,goal_avg,Shot_OnNet,Shot Score,Play,play_normalised,pass_avg,Faceoff Win,faceoff_win_normalised,faceoff_pct,Takeaway
44,Jillian Dempsey,0,0,1,1,3.0,0.0,0.75,0.0,0.629630,0.415344,14.0,0.777778,80.0,0.330579,0.450271,70.0,0.886076,0.721649,22.0
87,Mikyla Grant-Mentis,1,0,0,0,4.0,1.0,1.00,0.5,0.574074,0.695767,26.0,0.796296,104.0,0.429752,0.520736,19.0,0.240506,0.542857,19.0
98,Samantha Davis,1,0,0,0,3.0,1.0,0.75,0.5,0.605263,0.612782,19.0,0.684211,119.0,0.491736,0.565349,5.0,0.063291,0.625000,18.0
116,Taylor Woods,1,1,0,0,3.0,1.0,0.75,0.5,0.304348,0.577640,10.0,0.739130,157.0,0.648760,0.690307,2.0,0.025316,0.666667,14.0
119,Tereza Vanisova,0,1,1,0,0.0,0.0,0.00,0.0,0.466667,0.161905,14.0,0.666667,138.0,0.570248,0.646061,51.0,0.645570,0.662338,17.0


In [54]:
print('goal_avg_thres: ', goal_avg_thres)
print('pass_avg_thres: ', pass_avg_thres)
print('faceoff_avg_thres: ', faceoff_avg_thres)
print('takeaway_thres: ', takeaway_thres)

goal_avg_thres:  0.4387755102040816
pass_avg_thres:  0.5969175787357606
faceoff_avg_thres:  0.3845815752461322
takeaway_thres:  20.0


In [55]:
nwhl[(nwhl['Event'] == 'Play') & (nwhl['Player'] == 'Tereza Vanisova') & (nwhl['Player 2'] == 'Mikyla Grant-Mentis')]['Player'].count()

0

In [56]:
nwhl[(nwhl['Event'] == 'Incomplete Play') & (nwhl['Player'] == 'Tereza Vanisova') & (nwhl['Player 2'] == 'Mikyla Grant-Mentis')]['Player'].count()

0

### To find synergy between passers and other players

In [57]:
df_tmp = nwhl[['Team','Player','Player 2','Event']]

df2 = df_tmp[(df_tmp['Event'] == 'Play') | (df_tmp['Event']=='Incomplete Play')]
df2['Event'] = df2['Event'].apply(lambda x: 1 if(x=='Play') else 0)

df3 = df2.pivot_table(values = 'Event', index=['Team','Player','Player 2'], aggfunc={'count','mean','sum'})

df_syn = df3.sort_values(by=['mean','sum'],ascending=False)
df_syn.columns=['Total_passes','Success_rate','Successful_Passes']
df_syn

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Total_passes  ...  Successful_Passes
Team         Player                  Player 2                               ...                   
Boston Pride McKenna Brand           Kaleigh Fratkin                  35.0  ...               35.0
             Briana Mastel           Taylor Turnquist                 23.0  ...               23.0
Toronto Six  Taylor Woods            Lindsay Eastwood                 23.0  ...               23.0
                                     Sarah Steele                     17.0  ...               17.0
                                     Shiann Darkangelo                16.0  ...               16.0
...                                                                    ...  ...                ...
             Sarah Steele            Emma Greco                        1.0  ...                0.0
             Sarah-Eve Coutu Godbout Shiann Darkangelo                 1.0  ...                0.0
             Shiann Darkangelo       Natalie Marcuzzi                  1.0  ...                0.0
                                     Sarah-Eve Coutu Godbout           1.0  ...                0.0
             Taylor Woods            Emily Fluke                       8.0  ...                0.0

[1598 rows x 3 columns]

In [58]:
df_syn.reset_index(inplace=True)
df_syn.rename(columns = {'Player 2':'Player2'},inplace=True)
df_syn.head()

,Team,Player,Player2,Total_passes,Success_rate,Successful_Passes
0,Boston Pride,McKenna Brand,Kaleigh Fratkin,35.0,1.0,35.0
1,Boston Pride,Briana Mastel,Taylor Turnquist,23.0,1.0,23.0
2,Toronto Six,Taylor Woods,Lindsay Eastwood,23.0,1.0,23.0
3,Toronto Six,Taylor Woods,Sarah Steele,17.0,1.0,17.0
4,Toronto Six,Taylor Woods,Shiann Darkangelo,16.0,1.0,16.0


In [59]:
df_s1 = df_syn[df_syn['Player']=='Tereza Vanisova']
lst1 = ['Samantha Davis','Jillian Dempsey','Mikyla Grant-Mentis','Taylor Woods']
a1 = df_s1[df_s1.Player2.isin(lst1)]
a1

,Team,Player,Player2,Total_passes,Success_rate,Successful_Passes
323,Boston Pride,Tereza Vanisova,Jillian Dempsey,1.0,1.000000,1.0
782,Boston Pride,Tereza Vanisova,Samantha Davis,36.0,0.722222,26.0


#### Tereza has high synergy with Samantha and some with Jillian

In [60]:
df_s2 = df_syn[df_syn['Player']=='Taylor Woods']
lst2 = ['Samantha Davis','Jillian Dempsey','Mikyla Grant-Mentis','Tereza Vanisova']
a2 = df_s2[df_s2.Player2.isin(lst2)]
a2

,Team,Player,Player2,Total_passes,Success_rate,Successful_Passes
704,Toronto Six,Taylor Woods,Mikyla Grant-Mentis,35.0,0.771429,27.0


#### Taylor has good synergy with Mikyla

## Final Selection of 5 players:
1. Mikyla Grant-Mentis
2. Taylor Woods
3. Samantha Davis
4. Jillian Dempsey
5. Tereza Vanisova


# Thank You!
_________